## import modules and setup csv to be read

In [13]:
import pandas as pd
import os
import numpy as np

csvpath = os.path.join('..','Resources','purchase_data.csv')

hero =  pd.read_csv(csvpath)


## player count

In [14]:
## code snippit found online to clean the header names(remove spaces, lowercase, ect)
hero.columns = hero.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
## count of unique players
x = hero.sn.unique()
total_players = [len(x)]
# output to table
pd.DataFrame(total_players,columns=["Total Players"])


,Total Players
0,576


## purchasing analysis (total)

In [15]:
## keeping of variable full count
fullcount=len(x)

## output list intialization that will be appended to
y=[]
x=len(hero.item_id.unique())

## add unique item total to list
y.append(x)
## add average price to list
x=hero["price"].mean()
z='${:.2f}'.format(x)
y.append(z)
## add number of purchases to list
x=len(hero.index)
y.append(x)
## add total rev to list
x=hero["price"].sum()
z='${:.2f}'.format(x)
y.append(z)
## add list y to data frame 'output' and print
outputPA=pd.DataFrame(columns=["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"])
outputPA.loc[0] = y
outputPA


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## gender demographics

In [16]:
## output list intialization that will be appended to
male = []
female = []
other = []

## group by gender, then isolate only the males
gender = hero.groupby('gender')
Mgender=gender.get_group('Male')
## count the number of unique names in males, append to male list
male.append(len(Mgender.sn.unique()))
x='{:.2%}'.format(len(Mgender.sn.unique())/fullcount)
male.append(x)

## group by gender, then isolate only the females
gender = hero.groupby('gender')
Fgender=gender.get_group('Female')
## count the number of unique names in females, append to female list
female.append(len(Fgender.sn.unique()))
x='{:.2%}'.format(len(Fgender.sn.unique())/fullcount)
female.append(x)

## group by gender, then isolate only the others
gender = hero.groupby('gender')
Ogender=gender.get_group('Other / Non-Disclosed')
## count the number of unique names in others, append to others list
other.append(len(Ogender.sn.unique()))
x='{:.2%}'.format(len(Ogender.sn.unique())/fullcount)
other.append(x)

## add lists to output dataframe
outputGD=pd.DataFrame(columns=["Total Count","Percentage of Players"])
outputGD.rename(index={0:"Male",1:"Female",2:"Other / Non-Disclosed"})
outputGD.loc['Male'] = male
outputGD.loc['Female'] = female
outputGD.loc['Other / Non-Disclosed'] = other
outputGD


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## purchasing analysis (gender)

In [17]:
## output list intialization that will be appended to
male = []
female = []
other = []

## group by gender, then isolate only the females
gender = hero.groupby('gender')
Fgender=gender.get_group('Female')
female.append(len(Fgender))
female.append(Fgender["price"].mean())
female.append(Fgender["price"].sum())
f=len(Fgender.sn.unique())
female.append(Fgender["price"].sum()/f)

## group by gender, then isolate only the males
gender = hero.groupby('gender')
Mgender=gender.get_group('Male')
male.append(len(Mgender))
male.append(Mgender["price"].mean())
male.append(Mgender["price"].sum())
m=len(Mgender.sn.unique())
male.append(Mgender["price"].sum()/m)

## group by gender, then isolate only the others
gender = hero.groupby('gender')
Ogender=gender.get_group('Other / Non-Disclosed')
other.append(len(Ogender))
other.append(Ogender["price"].mean())
other.append(Ogender["price"].sum())
o=len(Ogender.sn.unique())
other.append(Ogender["price"].sum()/o)


outputPAG=pd.DataFrame(columns=["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"])
outputGD.rename(index={0:"Male",1:"Female",2:"Other / Non-Disclosed"})
outputPAG.loc['Female']=female
outputPAG.loc['Male']=male
outputPAG.loc['Other / Non-Disclosed']=other
outputPAG

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113.0,3.203009,361.94,4.468395
Male,652.0,3.017853,1967.64,4.065372
Other / Non-Disclosed,15.0,3.346000,50.19,4.562727


## age demographics (v1)

In [19]:
## take csv table and remove all duplicate 'names'
y=hero.drop_duplicates(['sn'])
## sort these unique name rows into bins based on age ranges
bins = pd.cut(y['age'],[0,9,14,19,24,29,34,39,100],labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+'])

y.groupby(bins)['age'].agg(['count'])


,count
age,
<10,17
10-14,22
15-19,107
20-24,258
25-29,77
30-34,52
35-39,31
40+,12


## age demographics (with percentages)

In [50]:
## get table with only one row per person
y=hero.drop_duplicates(['sn'])
## sort these rows into bins based off 'age'
bins = pd.cut(y['age'],[0,9,14,19,24,29,34,39,100],labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+'])

## function to perform calculations on these bins
def my_agg(x):
    names = {
        ## counts the number of obs in the specific bin
        'Total Count': x['age'].count(),
        ## calculate percentage of the the specific bin over full total of unique age obs
        'Percentage of Players':  x['age'].count()*100/y['age'].count(),
      }
    ## return the two columns 
    return pd.Series(names, index=['Total Count', 'Percentage of Players'])
## apply my function grouped by the bins i  created
y.groupby(bins).apply(my_agg)

,Total Count,Percentage of Players
age,,
<10,17.0,2.951389
10-14,22.0,3.819444
15-19,107.0,18.576389
20-24,258.0,44.791667
25-29,77.0,13.368056
30-34,52.0,9.027778
35-39,31.0,5.381944
40+,12.0,2.083333


## purchasing analysis (age)

In [75]:
##set y as full uncut csv table 
y=hero
## cut y into bins WITH duplicates
bins = pd.cut(y['age'],[0,9,14,19,24,29,34,39,100],labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+'])
## function to calculate various metrics on the data frame (WITH dupes)
def my_agg2(x):
    names = {
        'Purchase Count': x['age'].count(),
        'Average Purchase Price':  x['price'].sum()/x['price'].count(),
        'Total Purchase Value': x['price'].sum(),
        ## .nunique give number of unique obs of column given
        'Avg Total Purchase per Person': x['price'].sum()/x['sn'].nunique(),
      }

    return pd.Series(names, index=['Purchase Count', 'Average Purchase Price','Total Purchase Value','Avg Total Purchase per Person'])
##apply my agg 2 groups by the bins i created
y.groupby(bins).apply(my_agg2)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
age,,,,
<10,23.0,3.353478,77.13,4.537059
10-14,28.0,2.956429,82.78,3.762727
15-19,136.0,3.035956,412.89,3.858785
20-24,365.0,3.052219,1114.06,4.318062
25-29,101.0,2.900990,293.00,3.805195
30-34,73.0,2.931507,214.00,4.115385
35-39,41.0,3.601707,147.67,4.763548
40+,13.0,2.941538,38.24,3.186667


## top spenders

In [88]:
## set y as unaltered csv table
y=hero
## group all unique names together
y.groupby('sn')
##function to calculate metrics on these groups(individuals)
def my_agg3(x):
    names = {
        'Total Purchase Value': x['price'].sum(),
        'Purchase Count':x['sn'].count(),
        'Average Purchase Price':x['price'].sum()/x['sn'].count()
      }
    return pd.Series(names, index=['Purchase Count','Average Purchase Price','Total Purchase Value'])
## set z dataframe as the grouped y dataframe while adding the calculations for the individuals
z=y.groupby(['sn'],sort=False).apply(my_agg3)
## keep only the 5 highest total purchase values
z.nlargest(5,'Total Purchase Value')


,Purchase Count,Average Purchase Price,Total Purchase Value
sn,,,
Lisosia93,5.0,3.792000,18.96
Idastidru52,4.0,3.862500,15.45
Chamjask73,3.0,4.610000,13.83
Iral74,4.0,3.405000,13.62
Iskadarya95,3.0,4.366667,13.10


## most popular items

In [100]:
y=hero
y.groupby(['item_id','item_name'])

def my_agg4(x):
    names = {
        'Purchase Count':x['item_id'].count(),
        'Item Price':x['price'].sum()/x['price'].count(),
        'Total Purchase Value':x['price'].sum()*x['item_id'].count()/x['price'].count()
    }
    return pd.Series(names, index=['Purchase Count','Item Price','Total Purchase Value'])

z=y.groupby(['item_id','item_name'],sort=False).apply(my_agg4)
z.nlargest(5,'Purchase Count')


,,Purchase Count,Item Price,Total Purchase Value
item_id,item_name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12.0,4.23,50.76
108,"Extraction, Quickblade Of Trembling Hands",9.0,3.53,31.77
82,Nirvana,9.0,4.90,44.10
145,Fiery Glass Crusader,9.0,4.58,41.22
92,Final Critic,8.0,4.88,39.04


## most profitable items

In [99]:
y=hero
y.groupby(['item_id','item_name'])

def my_agg4(x):
    names = {
        'Purchase Count':x['item_id'].count(),
        'Item Price':x['price'].sum()/x['price'].count(),
        'Total Purchase Value':x['price'].sum()*x['item_id'].count()/x['price'].count()
    }
    return pd.Series(names, index=['Purchase Count','Item Price','Total Purchase Value'])

z=y.groupby(['item_id','item_name'],sort=False).apply(my_agg4)
z.nlargest(5,'Total Purchase Value')


,,Purchase Count,Item Price,Total Purchase Value
item_id,item_name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12.0,4.23,50.76
82,Nirvana,9.0,4.90,44.10
145,Fiery Glass Crusader,9.0,4.58,41.22
92,Final Critic,8.0,4.88,39.04
103,Singed Scalpel,8.0,4.35,34.80
